In [108]:
from datasets import load_dataset
from pprint import pprint
from collections import Counter
import pandas as pd
from datasets import Dataset
from unicodedata import normalize, is_normalized
from spacy_alignments.tokenizations import get_alignments
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, Trainer,BatchEncoding, pipeline
import torch
from seqeval.metrics.sequence_labeling import get_entities
import glob

In [2]:
dataset = load_dataset('llm-book/ner-wikipedia-dataset')

Using the latest cached version of the module from /root/.cache/huggingface/modules/datasets_modules/datasets/llm-book--ner-wikipedia-dataset/184bcf9be66116e777f2f534436226d47348676c93ba20cca58933f1b2b3b782 (last modified on Mon May 13 08:23:57 2024) since it couldn't be found locally at llm-book/ner-wikipedia-dataset, or remotely on the Hugging Face Hub.


In [3]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['curid', 'text', 'entities'],
        num_rows: 4274
    })
    validation: Dataset({
        features: ['curid', 'text', 'entities'],
        num_rows: 534
    })
    test: Dataset({
        features: ['curid', 'text', 'entities'],
        num_rows: 535
    })
})


In [4]:
pprint(list(dataset['train'])[:2])

[{'curid': '3638038',
  'entities': [{'name': 'さくら学院', 'span': [0, 5], 'type': 'その他の組織名'},
               {'name': 'Ciao Smiles', 'span': [6, 17], 'type': 'その他の組織名'}],
  'text': 'さくら学院、Ciao Smilesのメンバー。'},
 {'curid': '1729527',
  'entities': [{'name': 'レクレアティーボ・ウェルバ', 'span': [17, 30], 'type': 'その他の組織名'},
               {'name': 'プリメーラ・ディビシオン', 'span': [32, 44], 'type': 'その他の組織名'}],
  'text': '2008年10月5日、アウェーでのレクレアティーボ・ウェルバ戦でプリメーラ・ディビシオンでの初得点を決めた。'}]


In [5]:
for i in dataset['train']:
    pprint(i)
    break

{'curid': '3638038',
 'entities': [{'name': 'さくら学院', 'span': [0, 5], 'type': 'その他の組織名'},
              {'name': 'Ciao Smiles', 'span': [6, 17], 'type': 'その他の組織名'}],
 'text': 'さくら学院、Ciao Smilesのメンバー。'}


・今回はテキストに含まれる固有表現のスパンとそのタイプを指定する

In [6]:
# データセットの分析

def count_label_occurrences(dataset: Dataset) -> dict[str, int]:

    # 固有表現タイプを抽出したlistを作成する
    entities = [
        e['type'] for data in dataset for e in data['entities']
    ]

    # ラベルの出現回数が多い順に並び変える
    # Counterにはmost_common()メソッドがあり、(要素, 出現回数)という形のタプルを出現回数順に並べたリストを返す。
    label_counts = dict(Counter(entities).most_common())
    return label_counts


label_counts_dict = {}
for split in dataset:
    label_counts_dict[split] = count_label_occurrences(dataset[split])
df = pd.DataFrame(label_counts_dict)
df.loc['合計'] = df.sum()
df

,train,validation,test
人名,2394,299,287
法人名,2006,231,248
地名,1769,184,204
政治的組織名,953,121,106
製品名,934,123,158
施設名,868,103,137
その他の組織名,852,99,100
イベント名,831,85,93
合計,10607,1245,1333


In [7]:
def has_overlap(spans):
    sorted_spans = sorted(spans, key=lambda x: x[0])
    for i in range(1, len(sorted_spans)):
        if sorted_spans[i-1][1] > sorted_spans[i][0]:
            return 1
    return 0


overlap_count = 0
for split in dataset:
    for data in dataset[split]:
        if data['entities']:
            spans = [e['span'] for e in data['entities']]
            overlap_count += has_overlap(spans)

    print(f"{split}におけるスパンが重複する事例数：{overlap_count}")

trainにおけるスパンが重複する事例数：0
validationにおけるスパンが重複する事例数：0
testにおけるスパンが重複する事例数：0


In [8]:
spans

[[0, 9], [10, 21], [25, 37]]

In [9]:
data['entities']

[{'name': 'ダーヴラ・カーワン', 'span': [0, 9], 'type': '人名'},
 {'name': 'マーシー・ハーティガン', 'span': [10, 21], 'type': '人名'},
 {'name': 'ラッセル・T・デイヴィス', 'span': [25, 37], 'type': '人名'}]

In [10]:
dataset['test'][-1:]

{'curid': ['4113413'],
 'text': ['ダーヴラ・カーワンはマーシー・ハーティガンを演じ、ラッセル・T・デイヴィスは本作のポッドキャストコメンタリーで彼女について「これまでにないほどダークな悪役」と表現した。'],
 'entities': [[{'name': 'ダーヴラ・カーワン', 'span': [0, 9], 'type': '人名'},
   {'name': 'マーシー・ハーティガン', 'span': [10, 21], 'type': '人名'},
   {'name': 'ラッセル・T・デイヴィス', 'span': [25, 37], 'type': '人名'}]]}

## 前処理

## テキスト正規化

In [11]:
text = "ABCＡＢＣabcABCアイウｱｲｳ①②③123"

nomalized_text = normalize('NFKC', text)
print('正規化前', text)
print('正規化後', nomalized_text)

正規化前 ABCＡＢＣabcABCアイウｱｲｳ①②③123
正規化後 ABCABCabcABCアイウアイウ123123


In [12]:
count = 0
for split in dataset:
    for data in dataset[split]:
        if not is_normalized('NFKC',data['text']): # 正規化されていないとFalseをかえす？
            count += 1
print(f'正規化されていない事例数: {count}')

正規化されていない事例数: 0


In [13]:
text = "ABCＡＢＣabcABCアイウｱｲｳ①②③123"
is_normalized('NFKC', text)

False

In [14]:
# joinはイテレータを引数に受け取る => 文字を1文字ずつ処理
'/'.join(dataset['train'][0]['text'])

'さ/く/ら/学/院/、/C/i/a/o/ /S/m/i/l/e/s/の/メ/ン/バ/ー/。'

## 文字列とトークン列のアライメント

In [15]:
model_name = "cl-tohoku/bert-base-japanese-v3"
tokenizer = AutoTokenizer.from_pretrained(model_name)


text ='さくら学院は'

# 文字列のLISTに変換
characters = list(text)

# 特殊トークンも含めたリストにする
tokens = tokenizer.convert_ids_to_tokens(tokenizer.encode(text))

char_to_token_indices, token_to_char_indices = get_alignments(characters, tokens)
print(characters, tokens)
print('文字に対するトークンの位置',char_to_token_indices)
print('トークンに対する文字の位置',token_to_char_indices)

['さ', 'く', 'ら', '学', '院', 'は'] ['[CLS]', 'さくら', '学院', 'は', '[SEP]']
文字に対するトークンの位置 [[1], [1], [1], [2], [2], [3]]
トークンに対する文字の位置 [[], [0, 1, 2], [3, 4], [5], []]


In [16]:
print(characters, tokens)

['さ', 'く', 'ら', '学', '院', 'は'] ['[CLS]', 'さくら', '学院', 'は', '[SEP]']


In [17]:
text = '大谷翔平は岩手県水沢市出身'
entities = [
    {'name':'大谷正平', 'span':[0, 4], 'type':'人名'},
    {'name':'岩手県水沢市', 'span':[5, 11], 'type':'地方'}
]

In [18]:



def output_tokens_and_labels(text, entities, tokenizer):
    characters = list(text)
    tokens = tokenizer.convert_ids_to_tokens(tokenizer.encode(text))
    char_to_token_indices, _ = get_alignments(characters, tokens)
    
    # 0で初期化したラベルリスト
    labels = ['0'] * len(tokens)
    for entity in entities:
        entity_span, entity_type = entity['span'], entity['type']
        start = char_to_token_indices[entity_span[0]][0]
        end = char_to_token_indices[entity_span[1] -1][0]
        labels[start] = f"B-{entity_type}"
        for idx in range(start + 1, end + 1):
            labels[idx] = f"I-{entity_type}"
            print(labels)
    
        labels[0] = '-'
        labels[-1] = '-'
    return tokens, labels


tokens, labels = output_tokens_and_labels(text, entities, tokenizer)

df = pd.DataFrame({'トークン列':tokens, 'ラベル列':labels})
df.index.name = "位置"
df.T

['0', 'B-人名', 'I-人名', '0', '0', '0', '0', '0', '0', '0', '0']
['0', 'B-人名', 'I-人名', 'I-人名', '0', '0', '0', '0', '0', '0', '0']
['-', 'B-人名', 'I-人名', 'I-人名', '0', 'B-地方', 'I-地方', '0', '0', '0', '-']
['-', 'B-人名', 'I-人名', 'I-人名', '0', 'B-地方', 'I-地方', 'I-地方', '0', '0', '-']
['-', 'B-人名', 'I-人名', 'I-人名', '0', 'B-地方', 'I-地方', 'I-地方', 'I-地方', '0', '-']


位置,0,1,2,3,4,5,6,7,8,9,10
トークン列,[CLS],大谷,翔,##平,は,岩手,県,水沢,市,出身,[SEP]
ラベル列,-,B-人名,I-人名,I-人名,0,B-地方,I-地方,I-地方,I-地方,0,-


In [19]:
from typing import Any
from seqeval.metrics import classification_report

def create_character_labels(
    text: str, entities: list[dict[str, list[int] | str]]
) -> list[str]:
    """文字ベースでラベルのlistを作成"""
    # "O"のラベルで初期化したラベルのlistを作成する
    labels = ["O"] * len(text)
    for entity in entities: # 各固有表現を処理する
        entity_span, entity_type = entity["span"], entity["type"]
        # 固有表現の開始文字の位置に"B-"のラベルを設定する
        labels[entity_span[0]] = f"B-{entity_type}"
        # 固有表現の開始文字以外の位置に"I-"のラベルを設定する
        for i in range(entity_span[0] + 1, entity_span[1]):
            labels[i] = f"I-{entity_type}"
    return labels

def convert_results_to_labels(
    results: list[dict[str, Any]]
) -> tuple[list[list[str]], list[list[str]]]:
    """正解データと予測データのラベルのlistを作成"""
    true_labels, pred_labels = [], []
    for result in results: # 各事例を処理する
        # 文字ベースでラベルのリストを作成してlistに加える
        true_labels.append(
            create_character_labels(result["text"], result["entities"])
        )
        pred_labels.append(
            create_character_labels(result["text"], result["pred_entities"])
        )
    return true_labels, pred_labels

In [20]:
## 評価指標のseqevalの挙動

results = [
    {
        "text": "大谷翔平は岩手県水沢市出身",
        "entities": [
            {"name": "大谷翔平", "span": [0, 4], "type": "人名"},
            {"name": "岩手県水沢市", "span": [5, 11], "type": "地名"},
        ],
        "pred_entities": [
            {"name": "大谷翔平", "span": [0, 4], "type": "人名"},
            {"name": "岩手県", "span": [5, 8], "type": "地名"},
            {"name": "水沢市", "span": [8, 11], "type": "施設名"},
        ],
    }
]

true_labels, pred_labels = convert_results_to_labels(results)
print(classification_report(true_labels, pred_labels))

              precision    recall  f1-score   support

          人名       1.00      1.00      1.00         1
          地名       0.00      0.00      0.00         1
         施設名       0.00      0.00      0.00         0

   micro avg       0.33      0.50      0.40         2
   macro avg       0.33      0.33      0.33         2
weighted avg       0.50      0.50      0.50         2



/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
true_labels

[['B-人名',
  'I-人名',
  'I-人名',
  'I-人名',
  'O',
  'B-地名',
  'I-地名',
  'I-地名',
  'I-地名',
  'I-地名',
  'I-地名',
  'O',
  'O']]

In [22]:
from seqeval.metrics import f1_score, precision_score, recall_score

def compute_scores(true_labels: list[list[str]], pred_labels: list[list[str]], average:str) -> dict[str, float]:
    scores = {
        'precision': precision_score(true_labels, pred_labels, average=average),
        'recall': recall_score(true_labels, pred_labels, average=average),
        'F1-score': f1_score(true_labels, pred_labels, average=average),
    }

    return scores


print(compute_scores(true_labels, pred_labels, 'micro'))

{'precision': 0.3333333333333333, 'recall': 0.5, 'F1-score': 0.4}


## 固有表現認識モデルの実装

In [23]:
# BERTのファインチューニング

# label1id
def create_label2id(
    entities_list: list[list[dict[str, str | str]]]
) -> dict[str, int]:
    label2id = {"O": 0}

    # setなので重複はなし
    entity_type = set([e['type'] for entities in entities_list for e in entities])

    entity_types = sorted(entity_type)

    # 1entityにつき2種類登録
    for i, entity_type in enumerate(entity_types):
        label2id[f"B-{entity_type}"] = i*2 + 1
        label2id[f"I-{entity_type}"] = i*2 + 2
    return label2id


label2id = create_label2id(dataset['train']["entities"])
id2label = {id:v for v, id in label2id.items()}
pprint(id2label)

{0: 'O',
 1: 'B-その他の組織名',
 2: 'I-その他の組織名',
 3: 'B-イベント名',
 4: 'I-イベント名',
 5: 'B-人名',
 6: 'I-人名',
 7: 'B-地名',
 8: 'I-地名',
 9: 'B-政治的組織名',
 10: 'I-政治的組織名',
 11: 'B-施設名',
 12: 'I-施設名',
 13: 'B-法人名',
 14: 'I-法人名',
 15: 'B-製品名',
 16: 'I-製品名'}


In [24]:
# データの前処理


def preprocess_data(data, tokenizer, label2id) -> BatchEncoding:
    # トークナイゼーション
    inputs = tokenizer(data['text'], return_tensors='pt', return_special_tokens_mask=True)
    inputs = { k:v.squeeze(0) for k, v in inputs.items()}

    # textの文字リスト
    characters = list(data['text'])

    # 特殊トークンを含んだサブトークンリスト
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'])
    # print(tokens[1:-2])
    # print(characters)
    # print(tokens)

    # 文字に対するトークン位置を取得
    char_to_token_indeces, _ = get_alignments(characters, tokens)

    # トークンリストと同じ要素数の０値リストで初期化
    labels = torch.zeros_like(inputs['input_ids'])
    for entity in data['entities']:
        # print(char_to_token_indeces)
        # print(entity['span'][0])
        # print(entity['span'][1] - 1)

        # char_to_token_indecesのentityの最初と最後の位置を取得
        start_token_indeces = char_to_token_indeces[entity['span'][0]]

        # sliceは0からなので-1を加える
        end_token_indeces = char_to_token_indeces[entity['span'][1] - 1]
        # print(characters)
        # print(char_to_token_indeces)
        # print(entity['span'][0])
        # print(entity['span'][1] - 1)
        # print(start_token_indeces)
        # print(end_token_indeces)

        # 文字に対応するトークンが存在しなければスキップ 
        if(
            len(start_token_indeces) == 0
            or len(end_token_indeces) == 0
        ):
            # print(entity)
            # print(char_to_token_indeces[entity['span'][0]])
            continue

        start, end = start_token_indeces[0], end_token_indeces[0]
        # print(start, end)
        entity_type = entity['type']

        # トークン要素リストでラベル付け
        labels[start] = label2id[f"B-{entity_type}"]
        if start != end:
            labels[start + 1 : end + 1] = label2id[f"I-{entity_type}"]
        # print(labels)


    labels[torch.where(inputs["special_tokens_mask"])] = -100
    inputs['labels'] = labels
    return inputs
        
        

In [25]:
test = dataset['train'][0]
preprocess_data(test, tokenizer, label2id)

{'input_ids': tensor([    2, 16972, 14284,   384,    50, 13634,  7075, 20218, 18124,  7045,
           464, 12913,   385,     3]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'special_tokens_mask': tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 'labels': tensor([-100,    1,    2,    0,    1,    2,    2,    2,    2,    2,    0,    0,
            0, -100])}

In [26]:
pprint(dataset['train'][0])

{'curid': '3638038',
 'entities': [{'name': 'さくら学院', 'span': [0, 5], 'type': 'その他の組織名'},
              {'name': 'Ciao Smiles', 'span': [6, 17], 'type': 'その他の組織名'}],
 'text': 'さくら学院、Ciao Smilesのメンバー。'}


In [27]:
# 訓練セットに対して前処理を行う
train_dataset = dataset["train"].map(
    preprocess_data,
    fn_kwargs={
        "tokenizer": tokenizer,
        "label2id": label2id,
    },
    remove_columns=dataset["train"].column_names,
)
# 検証セットに対して前処理を行う
validation_dataset = dataset["validation"].map(
    preprocess_data,
    fn_kwargs={
        "tokenizer": tokenizer,
        "label2id": label2id,
    },
    remove_columns=dataset["validation"].column_names,
)

Parameter 'fn_kwargs'={'tokenizer': BertJapaneseTokenizer(name_or_path='cl-tohoku/bert-base-japanese-v3', vocab_size=32768, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}, 'label2id': {'O': 0, 'B-その他の組織名': 1, 'I-その他の組織名': 2, 'B-イベント名': 3, 'I-イベント名': 4

Map:   0%|          | 0/4274 [00:00<?, ? examples/s]

Map:   0%|          | 0/534 [00:00<?, ? examples/s]

In [28]:
from transformers import AutoModelForTokenClassification, DataCollatorForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(model_name, label2id=label2id, id2label=id2label)
data_collator = DataCollatorForTokenClassification(tokenizer)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
data_collator

DataCollatorForTokenClassification(tokenizer=BertJapaneseTokenizer(name_or_path='cl-tohoku/bert-base-japanese-v3', vocab_size=32768, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}, padding=True, max_length=None, pad_to_multiple_of=None, label_pad_toke

In [30]:
from transformers import Trainer, TrainingArguments
from transformers.trainer_utils import set_seed

set_seed(42)

training_args = TrainingArguments(
    output_dir='output_bert_ner',
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=1e-4,
    lr_scheduler_type='linear',
    warmup_ratio=0.1,
    num_train_epochs=5,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_strategy='epoch',
    fp16=True
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    data_collator=data_collator,
    args=training_args
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.678900,0.102102
2,0.067800,0.095260
3,0.028400,0.089854
4,0.012900,0.102608
5,0.006300,0.107572


TrainOutput(global_step=670, training_loss=0.1588394039602422, metrics={'train_runtime': 89.7939, 'train_samples_per_second': 237.989, 'train_steps_per_second': 7.462, 'total_flos': 1070012411245680.0, 'train_loss': 0.1588394039602422, 'epoch': 5.0})

In [31]:
from transformers import Trainer, TrainingArguments
from transformers.trainer_utils import set_seed

# 乱数シードを42に固定する
set_seed(42)

# Trainerに渡す引数を初期化する
training_args = TrainingArguments(
    output_dir="output_bert_ner", # 結果の保存フォルダ
    per_device_train_batch_size=32, # 訓練時のバッチサイズ
    per_device_eval_batch_size=32, # 評価時のバッチサイズ
    learning_rate=1e-4, # 学習率
    lr_scheduler_type="linear", # 学習率スケジューラ
    warmup_ratio=0.1, # 学習率のウォームアップ
    num_train_epochs=5, # 訓練エポック数
    evaluation_strategy="epoch", # 評価タイミング
    save_strategy="epoch", # チェックポイントの保存タイミング
    logging_strategy="epoch", # ロギングのタイミング
    fp16=True, # 自動混合精度演算の有効化
)

# Trainerを初期化する
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    data_collator=data_collator,
    args=training_args,
)

# 訓練する
trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.011800,0.140670
2,0.017000,0.134340
3,0.009200,0.123851
4,0.003800,0.136890
5,0.001600,0.139493


TrainOutput(global_step=670, training_loss=0.008705644398482877, metrics={'train_runtime': 95.0738, 'train_samples_per_second': 224.773, 'train_steps_per_second': 7.047, 'total_flos': 1070012411245680.0, 'train_loss': 0.008705644398482877, 'epoch': 5.0})

In [32]:
def convert_list_dict_to_dict_list(
    list_dict: dict[str, list]
) -> list[dict[str, list]]:
    """ミニバッチのデータを事例単位のlistに変換"""
    dict_list = []
    # dictのキーのlistを作成する
    keys = list(list_dict.keys())
    for idx in range(len(list_dict[keys[0]])): # 各事例で処理する
        # dictの各キーからデータを取り出してlistに追加する
        dict_list.append({key: list_dict[key][idx] for key in keys})
    return dict_list

# ミニバッチのデータを事例単位のlistに変換する
list_dict = {
    "input_ids": [[0, 1], [2, 3]],
    "labels": [[1, 2], [3, 4]],
}
dict_list = convert_list_dict_to_dict_list(list_dict)
print(f"入力: {list_dict}")
print(f"出力: {dict_list}")

入力: {'input_ids': [[0, 1], [2, 3]], 'labels': [[1, 2], [3, 4]]}
出力: [{'input_ids': [0, 1], 'labels': [1, 2]}, {'input_ids': [2, 3], 'labels': [3, 4]}]


In [33]:
 list(list_dict.keys())

['input_ids', 'labels']

In [34]:
from tqdm import tqdm
from torch.utils.data import DataLoader
from transformers import PreTrainedModel

def run_prediction(
    dataloader: DataLoader, model: PreTrainedModel
) -> list[dict[str, Any]]:
    """予測スコアに基づき固有表現ラベルを予測"""
    predictions = []
    for batch in tqdm(dataloader): # 各ミニバッチを処理する
        inputs = {
            k: v.to(model.device)
            for k, v in batch.items()
            if k != "special_tokens_mask"
        }
        # 予測スコアを取得する
        logits = model(**inputs).logits
        # 最もスコアの高いIDを取得する
        batch["pred_label_ids"] = logits.argmax(-1)
        batch = {k: v.cpu().tolist() for k, v in batch.items()}
        # ミニバッチのデータを事例単位のlistに変換する
        predictions += convert_list_dict_to_dict_list(batch)
    return predictions

# ミニバッチの作成にDataLoaderを用いる
validation_dataloader = DataLoader(
    validation_dataset,
    batch_size=32,
    shuffle=False,
    collate_fn=data_collator,
)
# 固有表現ラベルを予測する
predictions = run_prediction(validation_dataloader, model)
print(predictions[0]["pred_label_ids"])

100%|██████████| 17/17 [00:00<00:00, 25.31it/s]

[0, 0, 15, 16, 0, 0, 13, 14, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 13, 14, 14, 14, 13, 13, 0, 0, 0, 15, 15, 15, 16, 15, 13, 14, 14, 14, 16, 0, 13, 13, 14, 14, 14, 0, 0, 0, 13, 14, 14, 14, 13, 13, 14, 14, 14, 14, 15, 0, 0, 15, 15, 15, 16, 16, 0, 13, 14, 14, 14, 14, 13, 0, 0, 15, 15, 15, 16, 0, 0, 13, 14]


In [35]:
print(len(predictions))
for i in validation_dataloader:
    print(i['input_ids'].size())

534
torch.Size([32, 82])
torch.Size([32, 69])
torch.Size([32, 90])
torch.Size([32, 86])
torch.Size([32, 112])
torch.Size([32, 73])
torch.Size([32, 78])
torch.Size([32, 85])
torch.Size([32, 81])
torch.Size([32, 69])
torch.Size([32, 81])
torch.Size([32, 88])
torch.Size([32, 69])
torch.Size([32, 81])
torch.Size([32, 148])
torch.Size([32, 82])
torch.Size([22, 66])


In [36]:
for i in validation_dataloader:
    pprint(i['input_ids'].size())
    pprint(i)
    break

torch.Size([32, 82])
{'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]),
 'input_ids': tensor([[    2,   395, 13887,  ...,     0,     0,     0],
        [    2, 12538,   491,  ...,     0,     0,     0],
        [    2, 18763,  7087,  ...,     0,     0,     0],
        ...,
        [    2, 25912, 12881,  ...,     0,     0,     0],
        [    2, 12538,   500,  ...,     0,     0,     0],
        [    2,  1220, 12582,  ...,     0,     0,     0]]),
 'labels': tensor([[-100,    0,   15,  ..., -100, -100, -100],
        [-100,    0,    0,  ..., -100, -100, -100],
        [-100,    0,    0,  ..., -100, -100, -100],
        ...,
        [-100,   11,   12,  ..., -100, -100, -100],
        [-100,    0,    0,  ..., -100, -100, -100],
        [-100,    0,    0,  ..., -100, -100, -100]]),
 'special_tokens_mask': te

In [37]:
predictions[0]

{'input_ids': [2,
  395,
  13887,
  4436,
  396,
  465,
  14895,
  31227,
  7053,
  12488,
  464,
  12627,
  458,
  12493,
  456,
  12483,
  385,
  3,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'special_tokens_mask': [1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 

In [38]:

def extract_entites(predictions, dataset, tokenizer, id2label):
    """
    固有表現の抽出
    """
    results = []
    for prediction, data in zip(predictions, dataset):
    
        # 1文字のリスト化
        characters = list(data['text'])
        print('characters:',characters)
    
        tokens, pred_labels = [], []

        # idからトークンへ変換
        all_tokens = tokenizer.convert_ids_to_tokens(prediction['input_ids'])
    
        for token, label_id in zip(all_tokens, prediction['pred_label_ids']):
            
            # スペシャルトークンを除いたトークンリスト、予想ラベルリストを作成
            if token not in tokenizer.all_special_tokens:
                tokens.append(token)
                pred_labels.append(id2label[label_id])
    
    
            # スペシャルトークンを含まないトークンに対する文字の位置リストを返す
            _, token_to_char_indices = get_alignments(characters, tokens)


            
    
            pred_entities = []
            print('tokens:',tokens)
            print('pred_labels:', pred_labels)

            # token, label_idのイテレーション毎にpred_labelsを０からイテレーションを回す
            for entity in get_entities(pred_labels):
                print('===========get_entities=============')
                print('entity:', entity)
                
                print('token_to_char_indices:',token_to_char_indices)
                entity_type, token_start, token_end = entity
                char_start = token_to_char_indices[token_start][0]
                char_end = token_to_char_indices[token_end][-1] + 1
                pred_entity = {
                    'name': "".join(characters[char_start: char_end]), 
                    'span': [char_start, char_end],
                    'type': entity_type,
                }
                print('pred_entity:',pred_entity)
    
                pred_entities.append(pred_entity)

            # dataのバッチ毎に予想ラベルを追加する
            data['pred_entities'] = pred_entities
            results.append(data)
        return results


results = extract_entites(predictions, dataset['validation'], tokenizer, id2label)

            
            


characters: ['「', '復', '活', '篇', '」', 'は', 'グ', 'リ', 'ー', 'ン', 'バ', 'ニ', 'ー', 'か', 'ら', 'の', '発', '売', 'と', 'な', 'っ', 'て', 'い', 'る', '。']
tokens: []
pred_labels: []
tokens: ['「']
pred_labels: ['O']
tokens: ['「', '復活']
pred_labels: ['O', 'B-製品名']
===========get_entities=============
entity: ('製品名', 1, 1)
token_to_char_indices: [[0], [1, 2]]
pred_entity: {'name': '復活', 'span': [1, 3], 'type': '製品名'}
tokens: ['「', '復活', '篇']
pred_labels: ['O', 'B-製品名', 'I-製品名']
===========get_entities=============
entity: ('製品名', 1, 2)
token_to_char_indices: [[0], [1, 2], [3]]
pred_entity: {'name': '復活篇', 'span': [1, 4], 'type': '製品名'}
tokens: ['「', '復活', '篇', '」']
pred_labels: ['O', 'B-製品名', 'I-製品名', 'O']
===========get_entities=============
entity: ('製品名', 1, 2)
token_to_char_indices: [[0], [1, 2], [3], [4]]
pred_entity: {'name': '復活篇', 'span': [1, 4], 'type': '製品名'}
tokens: ['「', '復活', '篇', '」', 'は']
pred_labels: ['O', 'B-製品名', 'I-製品名', 'O', 'O']
===========get_entities=============
entity: ('製品名', 1, 

In [39]:
# ラストの０ラベルは0が続くのでエンティティとは認められない
get_entities( ['0', 'B-製品名', 'I-製品名', '0', '0', 'B-法人名', 'I-法人名', 'I-法人名', '0', '0', '0', '0', '0', '0', '0', '0'])

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


[('_', 0, 0), ('製品名', 1, 2), ('_', 3, 4), ('法人名', 5, 7)]

In [40]:
get_entities( ['O', 'B-製品名', 'I-製品名', 'O', 'O', 'B-法人名', 'I-法人名', 'I-法人名', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'])

[('製品名', 1, 2), ('法人名', 5, 7)]

In [41]:
from seqeval.metrics.sequence_labeling import get_entities
from transformers import PreTrainedTokenizer

def extract_entities(
    predictions: list[dict[str, Any]],
    dataset: list[dict[str, Any]],
    tokenizer: PreTrainedTokenizer,
    id2label: dict[int, str],
) -> list[dict[str, Any]]:
    """固有表現を抽出"""
    results = []
    for prediction, data in zip(predictions, dataset):
        # 文字のlistを取得する
        characters = list(data["text"])

        # 特殊トークンを除いたトークンのlistと予測ラベルのlistを取得する
        tokens, pred_labels = [], []
        all_tokens = tokenizer.convert_ids_to_tokens(
            prediction["input_ids"]
        )
        # print(prediction["pred_label_ids"])

        for token, label_id in zip(
            all_tokens, prediction["pred_label_ids"]
        ):
            # 特殊トークン以外をlistに追加する
            if token not in tokenizer.all_special_tokens:
                tokens.append(token)
                pred_labels.append(id2label[label_id])

        # 文字のlistとトークンのlistのアライメントをとる
        _, token_to_char_indices = get_alignments(characters, tokens)

        # 予測ラベルのlistから固有表現タイプと、
        # トークン単位の開始位置と終了位置を取得して、
        # それらを正解データと同じ形式に変換する
        pred_entities = []
        # print('pred_labels', pred_labels)
        for entity in get_entities(pred_labels):
            # print('entity:',entity)
            entity_type, token_start, token_end = entity
            # 文字単位の開始位置を取得する
            char_start = token_to_char_indices[token_start][0]
            # 文字単位の終了位置を取得する
            char_end = token_to_char_indices[token_end][-1] + 1
            pred_entity = {
                "name": "".join(characters[char_start:char_end]),
                "span": [char_start, char_end],
                "type": entity_type,
            }
            pred_entities.append(pred_entity)
        data["pred_entities"] = pred_entities
        results.append(data)

    return results

# 固有表現を抽出する
results = extract_entities(
    predictions, dataset["validation"], tokenizer, id2label
)
pprint(results[0])

{'curid': '1662110',
 'entities': [{'name': '復活篇', 'span': [1, 4], 'type': '製品名'},
              {'name': 'グリーンバニー', 'span': [6, 13], 'type': '法人名'}],
 'pred_entities': [{'name': '復活篇', 'span': [1, 4], 'type': '製品名'},
                   {'name': 'グリーンバニー', 'span': [6, 13], 'type': '法人名'}],
 'text': '「復活篇」はグリーンバニーからの発売となっている。'}


In [42]:
predictions[0]

{'input_ids': [2,
  395,
  13887,
  4436,
  396,
  465,
  14895,
  31227,
  7053,
  12488,
  464,
  12627,
  458,
  12493,
  456,
  12483,
  385,
  3,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'special_tokens_mask': [1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 

In [43]:
# モデルを読み込む
model_name = "llm-book/bert-base-japanese-v3-ner-wikipedia-dataset"
best_model = AutoModelForTokenClassification.from_pretrained(
    model_name
)
best_model = best_model.to("cuda:0")

In [44]:
# テストセットに対して前処理を行う
test_dataset = dataset["test"].map(
    preprocess_data,
    fn_kwargs={
        "tokenizer": tokenizer,
        "label2id": label2id,
    },
    remove_columns=dataset["test"].column_names,
)
# ミニバッチの作成にDataLoaderを用いる
test_dataloader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,
    collate_fn=data_collator,
)
# 固有表現ラベルを予測する
predictions = run_prediction(test_dataloader, best_model)
# 固有表現を抽出する
results = extract_entities(
    predictions, dataset["test"], tokenizer, id2label
)
# 正解データと予測データのラベルのlistを作成する
true_labels, pred_labels = convert_results_to_labels(results)
# 評価結果を出力する
print(classification_report(true_labels, pred_labels))

Map:   0%|          | 0/535 [00:00<?, ? examples/s]

100%|██████████| 17/17 [00:00<00:00, 20.04it/s]


              precision    recall  f1-score   support

     その他の組織名       0.86      0.83      0.84       100
       イベント名       0.82      0.92      0.87        93
          人名       0.96      0.96      0.96       287
          地名       0.85      0.86      0.86       204
      政治的組織名       0.75      0.89      0.81       106
         施設名       0.85      0.85      0.85       137
         法人名       0.89      0.88      0.89       248
         製品名       0.76      0.81      0.79       158

   micro avg       0.86      0.88      0.87      1333
   macro avg       0.84      0.88      0.86      1333
weighted avg       0.86      0.88      0.87      1333



In [45]:
results

[{'curid': '49396',
  'text': '統治機構の近代化により王朝を立て直すことに失敗、加えて義和団の乱後をめぐる清朝の醜態も加わり、1911年の辛亥革命への機運が高まる。',
  'entities': [{'name': '義和団の乱', 'span': [27, 32], 'type': 'イベント名'},
   {'name': '清朝', 'span': [37, 39], 'type': '政治的組織名'},
   {'name': '辛亥革命', 'span': [53, 57], 'type': 'イベント名'}],
  'pred_entities': [{'name': '義和団の乱', 'span': [27, 32], 'type': 'イベント名'},
   {'name': '清朝', 'span': [37, 39], 'type': '政治的組織名'},
   {'name': '辛亥革命', 'span': [53, 57], 'type': 'イベント名'}]},
 {'curid': '3352215',
  'text': '奏者は普通、爪を用いて演奏する。',
  'entities': [],
  'pred_entities': []},
 {'curid': '1755276',
  'text': "株式会社ナムコに入社し、ファミスタ'88等の作曲を手掛ける。",
  'entities': [{'name': '株式会社ナムコ', 'span': [0, 7], 'type': '法人名'},
   {'name': "ファミスタ'88", 'span': [12, 20], 'type': '製品名'}],
  'pred_entities': [{'name': '株式会社ナムコ', 'span': [0, 7], 'type': '法人名'},
   {'name': "ファミスタ'88", 'span': [12, 20], 'type': '製品名'}]},
 {'curid': '4019083',
  'text': '1947年、移行活動がまだ進行中であったため当時のペイン陸軍飛行場の軍事管理はアメリカ陸軍航空軍の後身であるアメリカ空軍に移管され、空港はペイン・フィールドと改名

In [46]:
a = {'entities': [{'name': '義和団の乱', 'span': [27, 32], 'type': 'イベント名'},
   {'name': '清朝', 'span': [37, 39], 'type': '政治的組織名'},
   {'name': '辛亥革命', 'span': [53, 57], 'type': 'イベント名'}]}

b = {'pred_entities': [{'name': '統治機構の近代化により王朝を立て直すことに失敗、加えて',
    'span': [0, 27],
    'type': '_'},
   {'name': '義和団の乱', 'span': [27, 32], 'type': 'イベント名'},
   {'name': '後をめぐる', 'span': [32, 37], 'type': '_'},
   {'name': '清朝', 'span': [37, 39], 'type': '政治的組織名'},
   {'name': 'の醜態も加わり、1911年の', 'span': [39, 53], 'type': '_'},
   {'name': '辛亥革命', 'span': [53, 57], 'type': 'イベント名'}]}

a['entities'] == b['pred_entities']

False

In [47]:
def find_error_results(
    results: list[dict[str, Any]],
) -> list[dict[str, Any]]:
    """エラー事例を発見"""
    error_results = []
    for idx, result in enumerate(results): # 各事例を処理する
        result["idx"] = idx
        # 正解データと予測データが異なるならばlistに加える
        if result["entities"] != result["pred_entities"]:
            error_results.append(result)
    return error_results

def output_text_with_label(result: dict[str, Any], entity_column: str) -> str:
    """固有表現ラベル付きテキストを出力"""
    text_with_label = ""
    entity_count = 0
    for i, char in enumerate(result["text"]): # 各文字を処理する
        # 出力に加えていない固有表現の有無を判定する
        if entity_count < len(result[entity_column]):
            entity = result[entity_column][entity_count]
            # 固有表現の先頭の処理を行う
            if i == entity["span"][0]:
                entity_type = entity["type"]
                text_with_label += f" [({entity_type}) "
            text_with_label += char
            # 固有表現の末尾の処理を行う
            if i == entity["span"][1] - 1:
                text_with_label += "] "
                entity_count += 1
        else:
            text_with_label += char
    return text_with_label

# エラー事例を発見する
error_results = find_error_results(results)
# 3件のエラー事例を出力する
for result in error_results[:3]:
    idx = result["idx"]
    true_text = output_text_with_label(result, "entities")
    pred_text = output_text_with_label(result, "pred_entities")
    print(f"事例{idx}の正解: {true_text}")
    print(f"事例{idx}の予測: {pred_text}")
    print()

事例18の正解:  [(法人名) 常盤木学園] 時代の同級生に [(その他の組織名) なでしこジャパン] の [(人名) 熊谷紗希] がいる。
事例18の予測:  [(施設名) 常盤木学園] 時代の同級生に [(その他の組織名) なでしこジャパン] の [(人名) 熊谷紗希] がいる。

事例19の正解: テレビで狼男映画の「 [(製品名) 倫敦の人狼] 」を見た [(人名) フィル・エヴァリー] は「ロンドンの狼男というタイトルで踊り騒げる曲を書いてみないか」と [(法人名) ジヴォン] に持ちかけた。
事例19の予測: テレビで狼男映画の「 [(製品名) 倫敦の人狼] 」を見た [(人名) フィル・エヴァリー] は「 [(製品名) ロンドン] の [(製品名) 狼男] というタイトルで踊り騒げる曲を書いてみないか」と [(人名) ジヴォン] に持ちかけた。

事例27の正解:  [(政治的組織名) 李承晩政権] 期から [(政治的組織名) 朴正煕政権] 期の1970年前後まで、南側の [(地名) 大韓民国] よりも北側の [(地名) 朝鮮民主主義人民共和国] の方が経済的な体力では勝っていたのである。
事例27の予測:  [(政治的組織名) 李]  [(人名) 承晩]  [(政治的組織名) 政権] 期から [(政治的組織名) 朴]  [(人名) 正煕]  [(政治的組織名) 政権] 期の1970年前後まで、南側の [(地名) 大韓民国] よりも北側の [(地名) 朝鮮民主主義人民共和国] の方が経済的な体力では勝っていたのである。



In [48]:
def create_transitions(label2id: dict[str, int]) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    """遷移スコアを定義"""
    # "B-"のラベルIDのlist
    b_ids = [v for k, v in label2id.items() if k[0] == "B"]
    # I-のラベルIDのlist
    i_ids = [v for k, v in label2id.items() if k[0] == "I"]
    o_id = label2id["O"]  # OのラベルID

    # 開始遷移スコアを定義する
    # すべてのスコアを-100で初期化する
    start_transitions = torch.full([len(label2id)], -100.0)
    # "B-"のラベルへ遷移可能として0を代入する
    start_transitions[b_ids] = 0
    # "O"のラベルへ遷移可能として0を代入する
    start_transitions[o_id] = 0

    # ラベル間の遷移スコアを定義する
    # すべてのスコアを-100で初期化する
    # # 行列（遷移前のラベルのid, 遷移後のラベルid）
    transitions = torch.full([len(label2id), len(label2id)], -100.0)
    # すべてのラベルから"B-"へ遷移可能として0を代入する
    transitions[:, b_ids] = 0
    # すべてのラベルから"O"へ遷移可能として0を代入する
    transitions[:, o_id] = 0
    # "B-"から同じタイプの"I-"へ遷移可能として0を代入する
    transitions[b_ids, i_ids] = 0
    # "I-"から同じタイプの"I-"へ遷移可能として0を代入する
    transitions[i_ids, i_ids] = 0

    # 終了遷移スコアを定義する
    # すべてのラベルから遷移可能としてすべてのスコアを0とする
    end_transitions = torch.zeros(len(label2id))
    return start_transitions, transitions, end_transitions

# 遷移スコアを定義する
start_transitions, transitions, end_transitions = create_transitions(
    label2id
)

In [49]:
label2id

{'O': 0,
 'B-その他の組織名': 1,
 'I-その他の組織名': 2,
 'B-イベント名': 3,
 'I-イベント名': 4,
 'B-人名': 5,
 'I-人名': 6,
 'B-地名': 7,
 'I-地名': 8,
 'B-政治的組織名': 9,
 'I-政治的組織名': 10,
 'B-施設名': 11,
 'I-施設名': 12,
 'B-法人名': 13,
 'I-法人名': 14,
 'B-製品名': 15,
 'I-製品名': 16}

In [50]:
 torch.full([len(label2id), len(label2id)], -100.0).size()

torch.Size([17, 17])

# ピタビアルゴリズムを用いたラベル列の予想

In [51]:
def decode_with_viterbi(
    emissions: torch.Tensor,  # ラベルの予測スコア
    mask: torch.Tensor,  # マスク
    start_transitions: torch.Tensor,  # 開始遷移スコア
    transitions: torch.Tensor,  # ラベル間の遷移スコア (ラベル, ラベル)
    end_transitions: torch.Tensor,  # 終了遷移スコア
) -> torch.Tensor:
    """ビタビアルゴリズムを用いて最適なラベル列を探索"""
    # バッチサイズと系列長を取得する
    batch_size, seq_length = mask.shape
    # 予測スコアとマスクに関して、0次元目と1次元目を入れ替える
    emissions = emissions.transpose(1, 0) # ====>　三次元では？(バッチ, 系列, ラベル) => (系列, バッチ, ラベル)
    mask = mask.transpose(1, 0) # (バッチ, 系列) => (系列, バッチ)

    histories = []  # 最適なラベル系列を保存するための履歴のlist
    # 開始遷移スコアと予測スコアを加算して、累積スコアの初期値とする
    score = start_transitions + emissions[0] # (系列, バッチ,ラベル) => 0系列の(バッチ,ラベル) 
    for i in range(1, seq_length):
        # 累積スコアを3次元に変換する
        broadcast_score = score.unsqueeze(2) # 0系列の(バッチ,ラベル) => (バッチ,ラベル, 1)
        # 現在の予測スコアを3次元に変換する
        broadcast_emission = emissions[i].unsqueeze(1)  # i系列の(バッチ,ラベル) => i系列の(バッチ, 1,ラベル)
        # 累積スコアと遷移スコアと現在の予測スコアを加算して、
        # 現在の累積スコアを取得する
        next_score = (
            # (バッチ,ラベル, 1) + (ラベル, ラベル) => (バッチ,ラベル,ラベル)
            # (バッチ,ラベル,ラベル) + i系列の(バッチ, 1,ラベル) => i系列までのスコアを算出
            broadcast_score + transitions + broadcast_emission
        )
        # 現在の累積スコアの各ラベルの最大値とそのインデックスを取得する
        next_score, indices = next_score.max(dim=1)
        # マスクしない要素の場合、累積スコアを更新する
        score = torch.where(mask[i].unsqueeze(1), next_score, score)
        # スコアの高いインデックスを履歴のlistに追加する
        histories.append(indices)
    # 終了遷移スコアを加算して合計スコアとする
    score += end_transitions

    # 各事例で最適なラベル列を取得する
    best_labels_list = []
    for i in range(batch_size):
        # 合計スコアの中で最大のスコアとなるラベルを取得する
        _, best_last_label = score[i].max(dim=0)
        best_labels = [best_last_label.item()]
        # 最後のラベルの遷移を逆方向に探索し、最適なラベル列を取得する
        for history in reversed(histories):
            best_last_label = history[i][best_labels[-1]]
            best_labels.append(best_last_label.item())
        # 順序を反転する
        best_labels.reverse()
        best_labels_list.append(best_labels)
    return torch.LongTensor(best_labels_list)

In [52]:
def run_prediction_viterbi(
    dataloader: DataLoader,
    model: PreTrainedModel,
) -> list[dict[str, Any]]:
    """ビダビアルゴリズムを用いてラベルを予測"""
    # 遷移スコアを取得する
    start_transitions, transitions, end_transitions = (
        create_transitions(model.config.label2id)
    )

    predictions = []
    for batch in tqdm(dataloader):  # 各ミニバッチを処理する
        inputs = {
            k: v.to(model.device)
            for k, v in batch.items()
            if k != "special_tokens_mask"
        }
        # [CLS]以外の予測スコアを取得する
        logits = model(**inputs).logits.cpu()[:, 1:, :]
        # [CLS]以外の特殊トークンのマスクを取得する
        mask = (batch["special_tokens_mask"].cpu() == 0)[:, 1:]
        # ビタビアルゴリズムを用いて最適なIDの系列を探索する
        pred_label_ids = decode_with_viterbi(
            logits,
            mask,
            start_transitions,
            transitions,
            end_transitions,
        )
        # [CLS]のIDを0とする
        cls_pred_label_id = torch.zeros(pred_label_ids.shape[0], 1)
        # [CLS]のIDと探索したIDの系列を連結して予測ラベルとする
        batch["pred_label_ids"] = torch.concat(
            [cls_pred_label_id, pred_label_ids], dim=1
        )
        batch = {k: v.cpu().tolist() for k, v in batch.items()}
        # ミニバッチのデータを事例単位のlistに変換する
        predictions += convert_list_dict_to_dict_list(batch)
    return predictions

# ビタビアルゴリズムを用いてラベルを予測する
predictions = run_prediction_viterbi(test_dataloader, best_model)
# 固有表現を抽出する
results = extract_entities(
    predictions, dataset["test"], tokenizer, id2label
)
# 正解データと予測データのラベルのlistを作成する
true_labels, pred_labels = convert_results_to_labels(results)
# 評価結果を出力する
print(classification_report(true_labels, pred_labels))

100%|██████████| 17/17 [00:01<00:00, 15.19it/s]


              precision    recall  f1-score   support

     その他の組織名       0.86      0.83      0.85       100
       イベント名       0.84      0.94      0.89        93
          人名       0.96      0.96      0.96       287
          地名       0.87      0.87      0.87       204
      政治的組織名       0.79      0.91      0.85       106
         施設名       0.88      0.86      0.87       137
         法人名       0.90      0.88      0.89       248
         製品名       0.79      0.81      0.80       158

   micro avg       0.88      0.89      0.88      1333
   macro avg       0.86      0.88      0.87      1333
weighted avg       0.88      0.89      0.88      1333



In [53]:
# logits, mask, start_transitions, transitions, end_transitionsの取得

def test(
    dataloader: DataLoader,
    model: PreTrainedModel,
) -> list[dict[str, Any]]:
    """ビダビアルゴリズムを用いてラベルを予測"""
    # 遷移スコアを取得する
    start_transitions, transitions, end_transitions = (
        create_transitions(model.config.label2id)
    )

    predictions = []
    for batch in tqdm(dataloader):  # 各ミニバッチを処理する
        inputs = {
            k: v.to(model.device)
            for k, v in batch.items()
            if k != "special_tokens_mask"
        }
        # [CLS]以外の予測スコアを取得する
        logits = model(**inputs).logits.cpu()[:, 1:, :]
        # [CLS]以外の特殊トークンのマスクを取得する
        mask = (batch["special_tokens_mask"].cpu() == 0)[:, 1:]
        # ビタビアルゴリズムを用いて最適なIDの系列を探索する
        pred_label_ids = decode_with_viterbi(
            logits,
            mask,
            start_transitions,
            transitions,
            end_transitions,
        )

    return logits, mask, start_transitions, transitions, end_transitions

In [54]:
logits, mask, start_transitions, transitions, end_transitions = test(test_dataloader, best_model)

100%|██████████| 17/17 [00:01<00:00, 14.85it/s]


In [55]:
logits.shape

torch.Size([23, 83, 17])

In [56]:
len(test_dataloader)

17

In [57]:
for i in test_dataloader:
    pprint(i['labels'].size())
    pprint(i)
    break

torch.Size([32, 70])
{'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]),
 'input_ids': tensor([[    2, 15233, 14227,  ...,     0,     0,     0],
        [    2, 18275,   465,  ...,     0,     0,     0],
        [    2, 12754, 12587,  ...,     0,     0,     0],
        ...,
        [    2, 18678,  7067,  ...,   449,   385,     3],
        [    2, 13888,  2002,  ...,     0,     0,     0],
        [    2, 17967, 12893,  ...,     0,     0,     0]]),
 'labels': tensor([[-100,    0,    0,  ..., -100, -100, -100],
        [-100,    0,    0,  ..., -100, -100, -100],
        [-100,   13,   14,  ..., -100, -100, -100],
        ...,
        [-100,   15,   16,  ...,    0,    0, -100],
        [-100,    0,    0,  ..., -100, -100, -100],
        [-100,    3,    4,  ..., -100, -100, -100]]),
 'special_tokens_mask': te

In [58]:
emissions = logits

batch_size, seq_length = mask.shape

emissions = emissions.transpose(1, 0) # ====>　三次元では？(バッチ, 系列, ラベル) => (系列, バッチ, ラベル)
mask = mask.transpose(1, 0) # (バッチ, 系列) => (系列, バッチ)

histories = []  

score = start_transitions + emissions[0] # (系列, バッチ,ラベル) => 0系列の(バッチ,ラベル) 
for i in range(1, seq_length):
    # 累積スコアを3次元に変換する
    broadcast_score = score.unsqueeze(2) # 0系列の(バッチ,ラベル) => (バッチ,ラベル, 1)
    # 現在の予測スコアを3次元に変換する
    broadcast_emission = emissions[i].unsqueeze(1)  # i系列の(バッチ,ラベル) => i系列の(バッチ, 1,ラベル)
    # 累積スコアと遷移スコアと現在の予測スコアを加算して、
    # 現在の累積スコアを取得する
    next_score = (
        # (バッチ,ラベル, 1) + (ラベル, ラベル) => (バッチ,ラベル,ラベル)
        # (バッチ,ラベル,ラベル) + i系列の(バッチ, 1,ラベル) => i系列までのスコアを算出
        broadcast_score + transitions + broadcast_emission
    )
    # 現在の累積スコアの各ラベルの最大値とそのインデックスを取得する
    next_score, indices = next_score.max(dim=1)
    # マスクしない要素の場合、累積スコアを更新する
    score = torch.where(mask[i].unsqueeze(1), next_score, score)
    # スコアの高いインデックスを履歴のlistに追加する
    histories.append(indices)
# 終了遷移スコアを加算して合計スコアとする
score += end_transitions

# # 各事例で最適なラベル列を取得する
best_labels_list = []
for i in range(batch_size):
    # 合計スコアの中で最大のスコアとなるラベルを取得する
    _, best_last_label = score[i].max(dim=0)
    best_labels = [best_last_label.item()]
    # 最後のラベルの遷移を逆方向に探索し、最適なラベル列を取得する
    for history in reversed(histories):
        best_last_label = history[i][best_labels[-1]]
        best_labels.append(best_last_label.item())
    # 順序を反転する
    best_labels.reverse()
    best_labels_list.append(best_labels)
torch.LongTensor(best_labels_list)

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [5, 6, 6,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [5, 6, 6,  ..., 0, 0, 0]])

In [100]:
histories[0]

tensor([[ 0,  0,  1,  0,  3,  0,  5,  0,  7,  0,  9,  0, 11,  0, 13,  0, 15],
        [ 0,  0,  1,  0,  3,  0,  5,  0,  7,  0,  9,  0, 11,  0, 13,  0, 15],
        [ 0,  0,  1,  0,  3,  0,  5,  0,  7,  0,  9,  0, 11,  0, 13,  0, 15],
        [13, 13,  1, 13,  3, 13,  5, 13,  7, 13,  9, 13, 11, 13, 13, 13, 15],
        [ 0,  0,  1,  0,  3,  0,  5,  0,  7,  0,  9,  0, 11,  0, 13,  0, 15],
        [ 0,  0,  1,  0,  3,  0,  5,  0,  7,  0,  9,  0, 11,  0, 13,  0, 15],
        [ 7,  7,  1,  7,  3,  7,  5,  7,  7,  7,  9,  7, 11,  7, 13,  7, 15],
        [ 0,  0,  1,  0,  3,  0,  5,  0,  7,  0,  9,  0, 11,  0, 13,  0, 15],
        [ 0,  0,  1,  0,  3,  0,  5,  0,  7,  0,  9,  0, 11,  0, 13,  0, 15],
        [ 5,  5,  1,  5,  3,  5,  5,  5,  7,  5,  9,  5, 11,  5, 13,  5, 15],
        [ 0,  0,  1,  0,  3,  0,  5,  0,  7,  0,  9,  0, 11,  0, 13,  0, 15],
        [ 0,  0,  1,  0,  3,  0,  5,  0,  7,  0,  9,  0, 11,  0, 13,  0, 15],
        [15, 15,  1, 15,  3, 15,  5, 15,  7, 15,  9, 15, 11, 15,

In [92]:
score.size()

torch.Size([23, 17])

In [59]:
matrix1 = torch.full((3, 5, 1), 10)
matrix2 = torch.full((5, 5), 2)

matrix1 + matrix2


tensor([[[12, 12, 12, 12, 12],
         [12, 12, 12, 12, 12],
         [12, 12, 12, 12, 12],
         [12, 12, 12, 12, 12],
         [12, 12, 12, 12, 12]],

        [[12, 12, 12, 12, 12],
         [12, 12, 12, 12, 12],
         [12, 12, 12, 12, 12],
         [12, 12, 12, 12, 12],
         [12, 12, 12, 12, 12]],

        [[12, 12, 12, 12, 12],
         [12, 12, 12, 12, 12],
         [12, 12, 12, 12, 12],
         [12, 12, 12, 12, 12],
         [12, 12, 12, 12, 12]]])

In [60]:
best_last_label

tensor(5)

In [61]:
def run_prediction_viterbi(
    dataloader: DataLoader,
    model: PreTrainedModel,
) -> list[dict[str, Any]]:
    """ビダビアルゴリズムを用いてラベルを予測"""
    # 遷移スコアを取得する
    start_transitions, transitions, end_transitions = (
        create_transitions(model.config.label2id)
    )

    predictions = []
    for batch in tqdm(dataloader):  # 各ミニバッチを処理する
        inputs = {
            k: v.to(model.device)
            for k, v in batch.items()
            if k != "special_tokens_mask"
        }
        # [CLS]以外の予測スコアを取得する
        logits = model(**inputs).logits.cpu()[:, 1:, :]
        # [CLS]以外の特殊トークンのマスクを取得する
        mask = (batch["special_tokens_mask"].cpu() == 0)[:, 1:]
        # ビタビアルゴリズムを用いて最適なIDの系列を探索する
        pred_label_ids = decode_with_viterbi(
            logits,
            mask,
            start_transitions,
            transitions,
            end_transitions,
        )
        # [CLS]のIDを0とする
        cls_pred_label_id = torch.zeros(pred_label_ids.shape[0], 1)
        # [CLS]のIDと探索したIDの系列を連結して予測ラベルとする
        batch["pred_label_ids"] = torch.concat(
            [cls_pred_label_id, pred_label_ids], dim=1
        )
        batch = {k: v.cpu().tolist() for k, v in batch.items()}
        # ミニバッチのデータを事例単位のlistに変換する
        predictions += convert_list_dict_to_dict_list(batch)
    return predictions

# ビタビアルゴリズムを用いてラベルを予測する
predictions = run_prediction_viterbi(test_dataloader, best_model)
# 固有表現を抽出する
results = extract_entities(
    predictions, dataset["test"], tokenizer, id2label
)
# 正解データと予測データのラベルのlistを作成する
true_labels, pred_labels = convert_results_to_labels(results)
# 評価結果を出力する
print(classification_report(true_labels, pred_labels))

100%|██████████| 17/17 [00:04<00:00,  3.72it/s]


              precision    recall  f1-score   support

     その他の組織名       0.86      0.83      0.85       100
       イベント名       0.84      0.94      0.89        93
          人名       0.96      0.96      0.96       287
          地名       0.87      0.87      0.87       204
      政治的組織名       0.79      0.91      0.85       106
         施設名       0.88      0.86      0.87       137
         法人名       0.90      0.88      0.89       248
         製品名       0.79      0.81      0.80       158

   micro avg       0.88      0.89      0.88      1333
   macro avg       0.86      0.88      0.87      1333
weighted avg       0.88      0.89      0.88      1333



In [87]:
histories[1].size()

torch.Size([23, 17])

In [69]:
[v for k, v in label2id.items() if k[0]=='B']

[1, 3, 5, 7, 9, 11, 13, 15]

535

In [103]:
from torchcrf import CRF
from transformers import BertForTokenClassification, PretrainedConfig
from transformers.modeling_outputs import TokenClassifierOutput

class BertWithCrfForTokenClassification(BertForTokenClassification):
    """BertForTokenClassificationにCRF層を加えたクラス：BertForTokenClassificationのオーバーライド""""

    def __init__(self, config: PretrainedConfig):
        """クラスの初期化"""
        super().__init__(config)
        # CRF層を定義する
        self.crf = CRF(len(config.label2id), batch_first=True)

    def _init_weights(self, module: torch.nn.Module) -> None:
        """定義した遷移スコアでパラメータを初期化"""
        super()._init_weights(module)
        if isinstance(module, CRF):
            st, t, et = create_transitions(self.config.label2id)
            module.start_transitions.data = st
            module.transitions.data = t
            module.end_transitions.data = et

    def forward(
        self,
        input_ids: torch.Tensor,
        attention_mask: torch.Tensor | None = None,
        token_type_ids: torch.Tensor | None = None,
        labels: torch.Tensor | None = None,
    ) -> TokenClassifierOutput:
        """モデルの前向き計算を定義"""
        # BertForTokenClassificationのforwardメソッドを適用して
        # 予測スコアを取得する
        output = super().forward(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        if labels is not None:
            logits = output.logits
            mask = labels != -100
            labels *= mask
            # 最終層のCRFによる損失を計算する
            output["loss"] = -self.crf(
                logits[:, 1:, :],
                labels[:, 1:],
                mask=mask[:, 1:],
                reduction="mean",
            )
        return output

# BertForTokenClassificationにCRF層を加えたクラスを定義する
model_crf = BertWithCrfForTokenClassification.from_pretrained(
    model_name, label2id=label2id, id2label=id2label
)

Some weights of BertWithCrfForTokenClassification were not initialized from the model checkpoint at llm-book/bert-base-japanese-v3-ner-wikipedia-dataset and are newly initialized: ['crf.end_transitions', 'crf.start_transitions', 'crf.transitions']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [104]:
# 乱数のシード値を再設定する
set_seed(42)

# Trainerに渡す引数を初期化する
training_args = TrainingArguments(
    output_dir="output_bert_crf_ner", # 結果の保存フォルダ
    per_device_train_batch_size=32, # 訓練時のバッチサイズ
    per_device_eval_batch_size=32, # 評価時のバッチサイズ
    learning_rate=1e-4, # 学習率
    lr_scheduler_type="linear", # 学習率スケジューラ
    warmup_ratio=0.1, # 学習率のウォームアップ
    num_train_epochs=5, # 訓練エポック数
    evaluation_strategy="epoch", # 評価タイミング
    save_strategy="epoch", # チェックポイントの保存タイミング
    logging_strategy="epoch", # ロギングのタイミング
    fp16=True, # 自動混合精度演算の有効化
)

# Trainerを初期化する
trainer = Trainer(
    model=model_crf,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    data_collator=data_collator,
    args=training_args,
)

# 訓練する
trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.421100,3.485203
2,0.478600,3.046370
3,0.299200,2.952504
4,0.205400,3.166773
5,0.150200,3.308664


TrainOutput(global_step=670, training_loss=0.3109138175622741, metrics={'train_runtime': 178.0368, 'train_samples_per_second': 120.031, 'train_steps_per_second': 3.763, 'total_flos': 1070016473991360.0, 'train_loss': 0.3109138175622741, 'epoch': 5.0})

In [105]:
def run_prediction_crf(
    dataloader: DataLoader,
    model: PreTrainedModel,
) -> list[dict[str, Any]]:
    """BERT-CRFモデルを用いてラベルを予測"""
    predictions = []
    for batch in tqdm(dataloader):  # 各ミニバッチを処理する
        inputs = {
            k: v.to(model.device)
            for k, v in batch.items()
            if k != "special_tokens_mask"
        }
        # [CLS]以外の予測スコアを取得する
        logits = model(**inputs).logits.cpu()[:, 1:, :]
        # [CLS]以外の特殊トークンのマスクを取得する
        mask = (batch["special_tokens_mask"] == 0).cpu()[:, 1:]
        # 訓練した遷移スコアを取得する
        # ================run\prediction_vitrbiからCRFによる追加・変更点====================
        start_transitions = model.crf.start_transitions.cpu()
        transitions = model.crf.transitions.cpu()
        end_transitions = model.crf.end_transitions.cpu()
        # ================================================================================

        
        # ビタビアルゴリズムを用いて最適なIDの系列を探索する
        pred_label_ids = decode_with_viterbi(
            logits,
            mask,
            start_transitions,
            transitions,
            end_transitions,
        )
        # [CLS]のIDを0とする
        cls_pred_label_id = torch.zeros(pred_label_ids.shape[0], 1)
        # [CLS]のIDと探索したIDの系列を連結して予測ラベルとする
        batch["pred_label_ids"] = torch.concat(
            [cls_pred_label_id, pred_label_ids], dim=1
        )
        batch = {k: v.cpu().tolist() for k, v in batch.items()}
        # ミニバッチのデータを事例単位のlistに変換する
        predictions += convert_list_dict_to_dict_list(batch)
    return predictions

In [115]:

best_score = 0
# 各チェックポイントで処理する
for checkpoint in sorted(glob.glob("./output_bert_crf_ner/checkpoint-*")):
    # モデルを読み込む
    model_crf = BertWithCrfForTokenClassification.from_pretrained(
        checkpoint
    )
    model_crf = model_crf.to("cuda:0")  # モデルをGPUに移動
    # 固有表現ラベルを予測する
    predictions = run_prediction_crf(validation_dataloader, model_crf)
    # 固有表現を抽出する
    results = extract_entities(
        predictions, dataset["validation"], tokenizer, id2label
    )
    # 正解データと予測データのラベルのlistを作成する
    true_labels, pred_labels = convert_results_to_labels(results)
    # 評価スコアを算出する
    scores = compute_scores(true_labels, pred_labels, "micro")
    if best_score < scores["F1-score"]:
        best_score = scores["F1-score"]
        best_model_crf = model_crf

100%|██████████| 17/17 [00:09<00:00,  1.78it/s]


In [117]:
print(scores)
print(best_model_crf)

{'precision': 0.9015212169735789, 'recall': 0.9044176706827309, 'F1-score': 0.9029671210906175}
BertWithCrfForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32768, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(i

In [119]:
best_model = best_model_crf.to('cuda:0') 

predictions = run_prediction_crf(test_dataloader, best_model_crf)
# 固有表現を抽出する
results = extract_entities(
    predictions, dataset["test"], tokenizer, id2label
)
# 正解データと予測データのラベルのlistを作成する
true_labels, pred_labels = convert_results_to_labels(results)
# 評価結果を出力する
print(classification_report(true_labels, pred_labels))

100%|██████████| 17/17 [00:14<00:00,  1.17it/s]


              precision    recall  f1-score   support

     その他の組織名       0.90      0.83      0.86       100
       イベント名       0.90      0.92      0.91        93
          人名       0.93      0.97      0.95       287
          地名       0.89      0.87      0.88       204
      政治的組織名       0.79      0.87      0.83       106
         施設名       0.88      0.86      0.87       137
         法人名       0.90      0.87      0.88       248
         製品名       0.83      0.84      0.83       158

   micro avg       0.89      0.89      0.89      1333
   macro avg       0.88      0.88      0.88      1333
weighted avg       0.89      0.89      0.89      1333



# 6.4 アノテーションツールを用いたデータセット構築

## 構築したデータセットでの性能評価

In [121]:
dataset_wikinews = load_dataset("llm-book/ner-wikinews-dataset")
print(dataset_wikinews)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for llm-book/ner-wikinews-dataset contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/llm-book/ner-wikinews-dataset
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['curid', 'text', 'entities'],
        num_rows: 178
    })
})


In [130]:
test_dataset_wikinews = dataset_wikinews["test"].map(
    preprocess_data,
    fn_kwargs={"tokenizer": tokenizer, "label2id": label2id},
    remove_columns=dataset_wikinews["test"].column_names
)

test_dataloader_wikinews = DataLoader(
    test_dataset_wikinews, batch_size=32, shuffle=False, collate_fn=data_collator
)

prediction = run_prediction_crf(test_dataloader_wikinews, best_model_crf)
results = extract_entities(
    prediction, dataset_wikinews["test"], tokenizer, id2label
)
true_labels, pred_labels = convert_results_to_labels(results)
print(classification_report(true_labels, pred_labels))


Map:   0%|          | 0/178 [00:00<?, ? examples/s]

100%|██████████| 6/6 [00:05<00:00,  1.15it/s]


              precision    recall  f1-score   support

     その他の組織名       0.89      0.67      0.76        12
       イベント名       0.73      0.64      0.68        25
          人名       0.95      0.98      0.97        62
          地名       0.85      0.87      0.86       143
      政治的組織名       0.77      0.94      0.84        49
         施設名       0.78      0.78      0.78        27
         法人名       0.90      0.80      0.85        56
         製品名       0.88      0.72      0.79        39

   micro avg       0.85      0.85      0.85       413
   macro avg       0.84      0.80      0.82       413
weighted avg       0.85      0.85      0.85       413



In [135]:
# エラー事例
error_results = find_error_results(results)
for result in error_results[:3]:
    idx = result['idx']
    true_text = output_text_with_label(result, "entities")
    pred_text = output_text_with_label(result, "pred_entities")
    print(f"事例{idx}の正解: {true_text}")
    print(f"事例{idx}の予測: {pred_text}")

事例1の正解: 2023年4月17日、 [(人名) 岸田裕子] 首相夫人は [(地名) アメリカ] の [(人名) バイデン] 大統領夫人の [(人名) ジル・バイデン] 夫人の招待を受けて、 [(施設名) ホワイトハウス] で会談した。
事例1の予測: 2023年4月17日、 [(人名) 岸田裕子] 首相夫人は [(地名) アメリカ] の [(人名) バイデン] 大統領夫人の [(人名) ジル・バイデン] 夫人の招待を受けて、 [(地名) ホワイトハウス] で会談した。
事例3の正解:  [(人名) 岸田] 夫人は、 [(人名) ジル] 夫人に2023年5月に [(地名) 広島] で開かれる [(イベント名) G7広島サミット] で、訪日してくれることを楽しみにしていると伝えた。
事例3の予測:  [(人名) 岸田] 夫人は、 [(人名) ジル] 夫人に2023年5月に [(地名) 広島] で開かれる [(イベント名) G7広島サミット] で、訪 [(地名) 日] してくれることを楽しみにしていると伝えた。
事例9の正解: 新庁舎は [(施設名) 旧京都府警本部] を改修した建物と新たに建設した建物となる。
事例9の予測: 新庁舎は [(政治的組織名) 旧京都府警本部] を改修した建物と新たに建設した建物となる。


In [134]:
error_results[0]

{'curid': '45869487',
 'text': '2023年4月17日、岸田裕子首相夫人はアメリカのバイデン大統領夫人のジル・バイデン夫人の招待を受けて、ホワイトハウスで会談した。',
 'entities': [{'name': '岸田裕子', 'span': [11, 15], 'type': '人名'},
  {'name': 'アメリカ', 'span': [20, 24], 'type': '地名'},
  {'name': 'バイデン', 'span': [25, 29], 'type': '人名'},
  {'name': 'ジル・バイデン', 'span': [35, 42], 'type': '人名'},
  {'name': 'ホワイトハウス', 'span': [52, 59], 'type': '施設名'}],
 'pred_entities': [{'name': '岸田裕子', 'span': [11, 15], 'type': '人名'},
  {'name': 'アメリカ', 'span': [20, 24], 'type': '地名'},
  {'name': 'バイデン', 'span': [25, 29], 'type': '人名'},
  {'name': 'ジル・バイデン', 'span': [35, 42], 'type': '人名'},
  {'name': 'ホワイトハウス', 'span': [52, 59], 'type': '地名'}],
 'idx': 1}